# Air Quality Prediction Project


This notebook merges weather and air quality data, trains regression and classification models,
compares their performance, and deploys the best model using Gradio.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, classification_report
import joblib
import gradio as gr

## Load and merge datasets

In [ ]:
# Load datasets
df_weather = pd.read_csv("weather_data_prepared.csv")
df_air = pd.read_csv("air_quality_prepared.csv")

# Convert datetime
df_weather["datetime"] = pd.to_datetime(df_weather["datetime"])
df_air["datetime"] = pd.to_datetime(df_air["datetime"])

# Merge on datetime
df_combined = pd.merge(df_air, df_weather, on="datetime", how="inner")

# Drop NA values
df_combined = df_combined.dropna(subset=["pm10", "temperature", "humidity", "wind_speed"])


## Feature Engineering

In [ ]:
def categorize_pm10(value):
    if value < 20:
        return "Low"
    elif 20 <= value <= 40:
        return "Moderate"
    else:
        return "High"

df_combined["pm10_category"] = df_combined["pm10"].apply(categorize_pm10)
X = df_combined[["temperature", "humidity", "wind_speed"]]
y_reg = df_combined["pm10"]
y_clf = df_combined["pm10_category"]

## Regression Modeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
r2_lr = r2_score(y_test, y_pred_lr)

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Linear Regression -> RMSE: {rmse_lr:.2f}, R^2: {r2_lr:.2f}")
print(f"Random Forest Regressor -> RMSE: {rmse_rf:.2f}, R^2: {r2_rf:.2f}")

## Classification Modeling

In [ ]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_clf, test_size=0.2, random_state=42)

log_clf = LogisticRegression(max_iter=1000, random_state=42)
log_clf.fit(X_train_c, y_train_c)
y_pred_log = log_clf.predict(X_test_c)
print("Logistic Regression Results:")
print(classification_report(y_test_c, y_pred_log))

rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_c, y_train_c)
y_pred_rf_clf = rf_clf.predict(X_test_c)
print("Random Forest Classifier Results:")
print(classification_report(y_test_c, y_pred_rf_clf))

## Save Final Model

In [ ]:
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X, y_reg)
joblib.dump(final_model, "final_rf_model.joblib")

## Gradio App

In [ ]:
model = joblib.load("final_rf_model.joblib")

def predict_pm10(temp, humidity, wind_speed):
    return float(model.predict([[temp, humidity, wind_speed]])[0])

demo = gr.Interface(
    fn=predict_pm10,
    inputs=[
        gr.Slider(-10, 40, value=20, label="Temperature (°C)"),
        gr.Slider(0, 100, value=50, label="Humidity (%)"),
        gr.Slider(0, 100, value=10, label="Wind Speed (km/h)")
    ],
    outputs=gr.Number(label="Predicted PM10 (µg/m³)"),
    title="PM10 Air Quality Predictor",
    description="Enter weather data to predict PM10 pollution level."
)

# Uncomment to run locally
# demo.launch()